In [7]:
from dotenv import load_dotenv
load_dotenv(override=True)
import os

from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel, set_tracing_disabled
set_tracing_disabled(True)

In [8]:
# 定义模型
BASE_URL = os.getenv("BASE_URL")
API_KEY = os.getenv("API_KEY")
MODEL = os.getenv("MODEL")

client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY
)

model = OpenAIChatCompletionsModel(
    openai_client=client,
    model=MODEL,
)

In [9]:
from pydantic import BaseModel

from agents import Agent

PROMPT = (
    "You are a helpful research assistant. Given a query, come up with a set of web searches "
    "to perform to best answer the query. Output between 5 and 20 terms to query for."
    "Your response should be in the form of JSON with the following schema: [{'reason': 'Your reasoning for why this search is important to the query.', 'query': 'The search term to use for the web search.'}]. Do not include any other text and do not wrap the response in a code block - just provide the raw json output."
)


class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""


planner_agent = Agent(
    name="PlannerAgent",
    instructions=PROMPT,
    model=model,
    output_type=WebSearchPlan,
)


In [12]:
from agents import Runner
query = "机器学习"
result = await Runner.run(
    planner_agent,
    f"Query: {query}",
)


In [21]:
async def _plan_searches(query: str) -> WebSearchPlan:
    print("planning", "Planning searches...")
    result = await Runner.run(
        planner_agent,
        f"Query: {query}",
    )
    return result

In [ ]:
result.final_output

WebSearchPlan(searches=[WebSearchItem(reason='了解机器学习的基本概念和定义，为后续深入研究打下基础。', query='机器学习 定义'), WebSearchItem(reason='探索机器学习在现实世界中的具体应用，帮助理解其重要性和用途。', query='机器学习 应用'), WebSearchItem(reason='确认常见的机器学习算法及其分类，帮助全面了解该领域。', query='机器学习 算法'), WebSearchItem(reason='掌握最新的机器学习发展趋势，有助于了解技术前沿。', query='机器学习 发展趋势'), WebSearchItem(reason='寻找高质量的学习资源，适合初学者或希望深入学习的人士。', query='机器学习 教程')])